In [1]:
import pandas as pd 
import random
  
from itertools import product
import numpy as np

from icecream import ic
from matplotlib import pyplot as plt
from itertools import accumulate
from scipy.spatial.distance import cdist 
from geopy.distance import geodesic
from itertools import combinations

In [2]:
# Set the variable to choose the dataset
country = "Italy"  # Change this to "China", "Russia", "US", or "Vanuatu" as needed

# Load the corresponding CSV file based on the country variable
cities = pd.read_csv(f'cities/{country.lower()}.csv', header=None, names=['City', 'lat', 'lon'])

DIST_MATRIX = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
cities
                                                                

,City,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34
5,Bolzano,46.50,11.35
6,Brescia,45.55,10.22
7,Cagliari,39.22,9.10
8,Catania,37.50,15.08
9,Ferrara,44.84,11.61


In [3]:
city_list = cities['City'].values.tolist()  # List of city names
city_indices = {city: index for index, city in enumerate(city_list)}
#If city_list is ['Rome', 'Milan', 'Naples'], then city_indices would be {'Rome': 0, 'Milan': 1, 'Naples': 2}.


In [4]:
#create a route with a greedy algorithm (start with a random city and then choice the nearest city)
def create_greedy_route(city_list, DIST_MATRIX, city_indices):
    """Create a route using a greedy approach, starting from a random city and visiting the nearest unvisited city."""
    # Step 1: Choose a random starting city
    start_city = random.choice(city_list)
    route = [start_city]
    unvisited = set(city_list) - {start_city} #set of unvisited cities

    # Step 2: Iteratively add the nearest unvisited city to the route
    current_city = start_city
    while unvisited:
        nearest_city = min(unvisited, key=lambda city: DIST_MATRIX[city_indices[current_city], city_indices[city]])
        route.append(nearest_city)
        unvisited.remove(nearest_city)
        current_city = nearest_city

    # Step 3: Close the loop by returning to the start city
    route.append(start_city)
    return route



def initial_population(pop_size, city_list, DIST_MATRIX, city_indices):
    """Generate an initial population of routes."""
    population = [create_greedy_route(city_list, DIST_MATRIX, city_indices) for _ in range(pop_size)]
    return population

#the cost needs to include the last step, from the last city back to the first city
#the number of steps corresponds to the number of cities
#the cost is the sum of the distances between the cities in the route

def route_distance(route, DIST_MATRIX, city_list):
    """Calculate the total distance of the route based on the distance matrix."""
    distance = 0
    for i in range(len(route)):
        start_city = route[i]
        end_city = route[(i + 1) % len(route)] #modulo to connect the last city with the first city #i+1 % len(route) -> i+1 % 5 = 1, 2, 3, 4, 0
        distance += DIST_MATRIX[city_list.index(start_city), city_list.index(end_city)]
    return distance

In [5]:
# Parameters
pop_size = 2
max_generations = 1000  # Define how many generations to run

city_list= cities['City'].values.tolist() #only the names of the cities. cities['City'].values.tolist()

##Now try to create a population of two routes
population = initial_population(pop_size, city_list, DIST_MATRIX, city_indices) #2 parents
print("Initial population", population)
routeDistance0= route_distance(population[0], DIST_MATRIX, city_list)
print("Route Distance 0:", routeDistance0)
routeDistance1= route_distance(population[1], DIST_MATRIX, city_list)
print("Route Distance 1:", routeDistance1)




Initial population [['Piacenza', 'Parma', "Reggio nell'Emilia", 'Modena', 'Bologna', 'Ferrara', 'Padua', 'Vicenza', 'Verona', 'Brescia', 'Bergamo', 'Monza', 'Milan', 'Novara', 'Turin', 'Genoa', 'Leghorn', 'Prato', 'Florence', 'Forlì', 'Ravenna', 'Rimini', 'Ancona', 'Perugia', 'Terni', 'Rome', 'Latina', 'Giugliano in Campania', 'Naples', 'Salerno', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Messina', 'Reggio di Calabria', 'Catania', 'Syracuse', 'Palermo', 'Cagliari', 'Sassari', 'Pescara', 'Trieste', 'Venice', 'Trento', 'Bolzano', 'Piacenza'], ['Turin', 'Novara', 'Milan', 'Monza', 'Bergamo', 'Brescia', 'Verona', 'Vicenza', 'Padua', 'Venice', 'Ferrara', 'Bologna', 'Modena', "Reggio nell'Emilia", 'Parma', 'Piacenza', 'Genoa', 'Leghorn', 'Prato', 'Florence', 'Forlì', 'Ravenna', 'Rimini', 'Ancona', 'Perugia', 'Terni', 'Rome', 'Latina', 'Giugliano in Campania', 'Naples', 'Salerno', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Messina', 'Reggio di Calabria', 'Catania', 'Syracuse', 'Palermo', 'Cagliari

In [6]:
#before mutation (inversion mutation) and then crossover (inver over)

# Start with an "inversion mutation":
# Select Two Points in the route randomly. These two points will define the segment of the route to be reversed.
# Reverse the Segment between these points.
# Return the Mutated Route.
def inversion_mutation(route):
    """Apply inversion mutation to a route by reversing a random segment of the route."""
    # Step 1: Choose two random indices in the route
    start, end = sorted(random.sample(range(len(route)), 2))
    
    # Step 2: Reverse the cities between the two indices
    route[start:end+1] = reversed(route[start:end+1])
    
    return route


In [7]:
#then we apply the inver over crossover method
#The goal of this method is to combine two parent routes while trying to preserve the sequence of edges from one parent as much as possible, by performing an inversion mutation that integrates edges from the second parent.

def inver_over_crossover(parent1, parent2):
    """Perform Inver Over crossover between two parent routes."""
    child = parent1[:-1]  # Exclude the last city to avoid duplication, when later appending the starting city at the end of the child route.
    current_index = random.randint(0, len(child) - 2) #a random index is selected from the child route to start the crossover process
    
    while True: #This loop continues until the entire route has been processed.
        current_city = child[current_index] #The current_city is determined from the child using current_index.
        next_city_in_parent2 = get_next_city_in_parent(parent2, current_city) #The next city in parent2 after current_city is obtained using the helper function get_next_city_in_parent.

        if next_city_in_parent2 == child[(current_index + 1) % len(child)]: #If the next_city_in_parent2 is the same as the next city in the child, the algorithm continues to the next index. This avoids duplicating edges in the child route.
            current_index = (current_index + 1) % (len(child) - 1)
            if current_index == 0:
                break
            continue
        
        target_index = child.index(next_city_in_parent2) #The index of next_city_in_parent2 in the child is found. This index will be used to determine where to perform the inversion mutation.
        if target_index > current_index: #If target_index is greater than current_index, it means the segment between these two indices can be reversed:
            child[current_index + 1:target_index + 1] = reversed(child[current_index + 1:target_index + 1]) 
        else: #If target_index is less than current_index, it indicates that the segment wraps around the end of the list, and that segment is reversed:
            sub_segment = child[current_index + 1:] + child[:target_index + 1] #The sub_segment is created by concatenating the two segments.
            sub_segment = list(reversed(sub_segment)) #The sub_segment is reversed.
            child[current_index + 1:], child[:target_index + 1] = sub_segment[:len(child) - current_index - 1], sub_segment[len(child) - current_index - 1:] #The reversed sub_segment is split back into two segments and assigned to the child route.

        current_index = (current_index + 1) % (len(child) - 1) #Move to the next index for the next iteration. The modulo operation ensures it wraps around.
        if current_index == 0:
            break

    child.append(child[0])  # Close the route by adding the starting city at the end
    return child

def get_next_city_in_parent(parent, city):
    """Get the next city in a parent route after the specified city, wrapping around if necessary."""
    idx = parent.index(city)
    return parent[(idx + 1) % len(parent)]

# Multiple Iterations to Adjust the Route: Each iteration in the while-loop either:
# Moves to the next city if the next edge already exists in the child, or Performs a reversal if the edge from parent2 is missing in the child.
# By repeating the check across multiple cities, the child route can incorporate as many edges as possible from parent2, effectively combining elements from both parents.


In [8]:
#the idea is to generate a child from the two parents and then replace one of the parents or both with the child if the child is better than the parent. 

# Initialize variables to track the best solution
best_route = None
best_distance = float('inf')

# Start evolving the population
for generation in range(max_generations):
    print(f"\n--- Generation {generation + 1} ---")
    
    # Evaluate the current population
    route_distances = [route_distance(route, DIST_MATRIX, city_list) for route in population]
    
    # Find the best route in the current population
    for i, route in enumerate(population):
        distance = route_distances[i]
        print(f"Route {i}: {route} | Distance: {distance}")
        
        # Update the best route if this one is better
        if distance < best_distance:
            best_distance = distance
            best_route = route
    
    # Apply inversion mutation on both two routes
    mutated_route = inversion_mutation(population[0].copy()) 
    mutated_route_distance = route_distance(mutated_route, DIST_MATRIX, city_list)
    print("Mutated Route:", mutated_route)
    print("Mutated Route Distance:", mutated_route_distance)

    if mutated_route_distance < route_distances[0]: # Replace the original route if the mutated one is better
        print("The mutated route is better than the original route.")
        population[0] = mutated_route

    mutated_route = inversion_mutation(population[1].copy()) 
    mutated_route_distance = route_distance(mutated_route, DIST_MATRIX, city_list)
    print("Mutated Route:", mutated_route)
    print("Mutated Route Distance:", mutated_route_distance)

    if mutated_route_distance < route_distances[1]: 
        print("The mutated route is better than the original route.")
        population[1] = mutated_route
    
    # Apply crossover to create a child route
    child = inver_over_crossover(population[0], population[1])
    child_distance = route_distance(child, DIST_MATRIX, city_list)
    print("Child Route:", child)
    print("Child Route Distance:", child_distance)


    # Replace one of the parent routes with the child if the child is better
    if child_distance < min(route_distances):
        index = route_distances.index(max(route_distances))
        population[index] = child
        print("The child route is better than one of the parent routes.")
        
# Print the best solution found
print("\n--- Best Solution Found ---")
print("Best Route:", best_route)
print("Best Route Distance:", best_distance)
# larger max_generation value leads to better results
#final cost is the distance of the best route found. number of steps corresponds to the number of max_generations.



--- Generation 1 ---
Route 0: ['Piacenza', 'Parma', "Reggio nell'Emilia", 'Modena', 'Bologna', 'Ferrara', 'Padua', 'Vicenza', 'Verona', 'Brescia', 'Bergamo', 'Monza', 'Milan', 'Novara', 'Turin', 'Genoa', 'Leghorn', 'Prato', 'Florence', 'Forlì', 'Ravenna', 'Rimini', 'Ancona', 'Perugia', 'Terni', 'Rome', 'Latina', 'Giugliano in Campania', 'Naples', 'Salerno', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Messina', 'Reggio di Calabria', 'Catania', 'Syracuse', 'Palermo', 'Cagliari', 'Sassari', 'Pescara', 'Trieste', 'Venice', 'Trento', 'Bolzano', 'Piacenza'] | Distance: 4576.195155974608
Route 1: ['Turin', 'Novara', 'Milan', 'Monza', 'Bergamo', 'Brescia', 'Verona', 'Vicenza', 'Padua', 'Venice', 'Ferrara', 'Bologna', 'Modena', "Reggio nell'Emilia", 'Parma', 'Piacenza', 'Genoa', 'Leghorn', 'Prato', 'Florence', 'Forlì', 'Ravenna', 'Rimini', 'Ancona', 'Perugia', 'Terni', 'Rome', 'Latina', 'Giugliano in Campania', 'Naples', 'Salerno', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Messina', 'Reggio di Calab

Algorithm's Performance:

-Greedy Algorithm:
My current implementation employs a greedy algorithm to create initial routes. Greedy algorithms are generally faster than exhaustive search methods since they construct solutions step-by-step, choosing the locally optimal solution at each stage.
However, greedy algorithms may not always yield the optimal solution for TSP, especially as the number of cities increases. They can quickly become trapped in local optima.

-Genetic Algorithm (GA):
The part of my code that includes the mutation and crossover processes is characteristic of genetic algorithms. GAs are typically slower than greedy algorithms because they involve multiple iterations, evaluating a population of solutions over many generations. They explore a broader solution space but may take longer to converge to an optimal or near-optimal solution.